## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import matplotlib.pyplot as plt
import requests
import numpy as np
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Elizabeth,ZA,-33.9180,25.5701,54.10,87,81,5.23,broken clouds
1,1,Bethel,US,41.3712,-73.4140,66.04,89,93,3.65,overcast clouds
2,2,Dingle,PH,10.9995,122.6711,88.81,61,75,9.48,broken clouds
3,3,Tuktoyaktuk,CA,69.4541,-133.0374,43.18,71,80,11.01,broken clouds
4,4,Faanui,PF,-16.4833,-151.7500,80.58,78,66,20.04,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Dingle,PH,10.9995,122.6711,88.81,61,75,9.48,broken clouds
4,4,Faanui,PF,-16.4833,-151.7500,80.58,78,66,20.04,light rain
5,5,Taitung,TW,22.7583,121.1444,89.87,62,75,12.66,broken clouds
6,6,Rikitea,PF,-23.1203,-134.9692,75.04,80,100,11.03,overcast clouds
7,7,Kahului,US,20.8947,-156.4700,76.96,85,98,10.76,overcast clouds
8,8,Garut,ID,-7.2024,107.8878,79.86,54,20,2.75,moderate rain
14,14,Pecos,US,31.4229,-103.4932,80.82,44,1,10.36,clear sky
19,19,Butaritari,KI,3.0707,172.7902,83.39,76,21,14.65,light rain
26,26,Hilo,US,19.7297,-155.0900,79.29,93,98,6.06,light rain
28,28,Dawlatabad,AF,36.4141,64.9053,85.06,9,0,8.46,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                222
City                   222
Country                222
Lat                    222
Lng                    222
Max Temp               222
Humidity               222
Cloudiness             222
Wind Speed             222
Current Description    222
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Dingle,PH,10.9995,122.6711,88.81,61,75,9.48,broken clouds
4,4,Faanui,PF,-16.4833,-151.7500,80.58,78,66,20.04,light rain
5,5,Taitung,TW,22.7583,121.1444,89.87,62,75,12.66,broken clouds
6,6,Rikitea,PF,-23.1203,-134.9692,75.04,80,100,11.03,overcast clouds
7,7,Kahului,US,20.8947,-156.4700,76.96,85,98,10.76,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
679,679,Baripada,IN,21.9350,86.7214,88.54,60,100,3.60,moderate rain
680,680,San Mateo Del Mar,MX,16.2000,-95.0000,80.28,79,100,13.13,overcast clouds
681,681,Puerto Lempira,HN,15.2667,-83.7667,77.04,92,96,7.07,overcast clouds
687,687,Strezhevoy,RU,60.7333,77.5889,75.18,46,56,10.29,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df[["Hotel Name"]] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Dingle,PH,88.81,10.9995,122.6711,
4,Faanui,PF,80.58,-16.4833,-151.7500,
5,Taitung,TW,89.87,22.7583,121.1444,
6,Rikitea,PF,75.04,-23.1203,-134.9692,
7,Kahului,US,76.96,20.8947,-156.4700,
8,Garut,ID,79.86,-7.2024,107.8878,
14,Pecos,US,80.82,31.4229,-103.4932,
19,Butaritari,KI,83.39,3.0707,172.7902,
26,Hilo,US,79.29,19.7297,-155.0900,
28,Dawlatabad,AF,85.06,36.4141,64.9053,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
       
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
df1 = hotel_df[hotel_df["Hotel Name"]!=""]
df1

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Dingle,PH,88.81,10.9995,122.6711,SEGAYA PROPERTY
4,Faanui,PF,80.58,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
5,Taitung,TW,89.87,22.7583,121.1444,Taitung Bali Suites Hotel
6,Rikitea,PF,75.04,-23.1203,-134.9692,People ThankYou
7,Kahului,US,76.96,20.8947,-156.4700,Maui Seaside Hotel
...,...,...,...,...,...,...
678,Yarada,IN,88.12,17.6500,83.2667,Jeevan
679,Baripada,IN,88.54,21.9350,86.7214,Aparna Hotel
681,Puerto Lempira,HN,77.04,15.2667,-83.7667,Hotel Yu baiwan
687,Strezhevoy,RU,75.18,60.7333,77.5889,Sibiryachka


In [14]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace = True)


In [15]:
# 8a. Create the output File (CSV)
hotel_df.to_csv('WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
city_data_df.to_csv(r'C:\Users\wendy\Desktop\GITHub\World_Weather_Analysis\Vacation_Search\city_data.csv', index_label='City_ID')

In [16]:
hotel_df


,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Dingle,PH,88.81,10.9995,122.6711,SEGAYA PROPERTY
4,Faanui,PF,80.58,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
5,Taitung,TW,89.87,22.7583,121.1444,Taitung Bali Suites Hotel
6,Rikitea,PF,75.04,-23.1203,-134.9692,People ThankYou
7,Kahului,US,76.96,20.8947,-156.4700,Maui Seaside Hotel
...,...,...,...,...,...,...
679,Baripada,IN,88.54,21.9350,86.7214,Aparna Hotel
680,San Mateo Del Mar,MX,80.28,16.2000,-95.0000,
681,Puerto Lempira,HN,77.04,15.2667,-83.7667,Hotel Yu baiwan
687,Strezhevoy,RU,75.18,60.7333,77.5889,Sibiryachka


In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [28]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in preferred_cities_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations_df = hotel_df[['Lat', 'Lng']]


KeyError: 'Hotel Name'

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = locations_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))